In [1]:
!pip install pandas
!pip install scikit-learn
!pip install nltk
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 53.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 82.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 69.8 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 789.3/789.3 kB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.1 MB/s eta 0:00:0000:01


In [2]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import sklearn as sk
import sklearn.model_selection

import tiktoken

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def tokenizer(X):
    stop_words = set(stopwords.words('english'))
    num_tokens = 0
    num_sentences = 0
    tokenized_documents = []
    lemmatizer = WordNetLemmatizer()
    for text in X:
        sentences = sent_tokenize(text)
        num_sentences += len(sentences)
        tokenized_sentences = [nltk.RegexpTokenizer(r"\w+").tokenize(s) for s in sentences]
        tokenized_document = [lemmatizer.lemmatize(word.lower()) for sentence in tokenized_sentences for word in sentence if word.lower() not in stop_words]
        num_tokens += len(tokenized_document)
        tokenized_documents.append(tokenized_document)
    print("Number of tokens: ", num_tokens)
    print("Number of sentences: ", num_sentences)
    return tokenized_documents


import sklearn as sk

# tf-idf vectorizer
def vectorizer(tokenized_documents):
    ## tokenized_documents is a list of lists, where each inner list contains tokens
    vectorizer = sk.feature_extraction.text.TfidfVectorizer(lowercase=False, preprocessor=None, tokenizer=lambda x: x)
    X = vectorizer.fit_transform(tokenized_documents)
    return X, vectorizer

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/onyxia/nltk_data...


# NLTK TOKENIZER AND TF-IDF VECTORIZER

In [3]:
import sys
# sys.path.append('../')
import numpy as np
import pandas as pd
import sklearn as sk

In [4]:
data = pd.read_csv('../../_data/Reviews.csv') # Loading the dataset
X, y = data['Text'], data['Score']

In [5]:
# FOR TESTING : only select first 20000 samples
# X, y = X[:20000], y[:20000]

In [6]:
tokenized_documents = tokenizer(X)

Number of tokens:  23767229
Number of sentences:  2832806


In [7]:
X, vect = vectorizer(tokenized_documents)

/opt/mamba/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [8]:
# most frequent words
print("Top 10 most frequent words in the dataset")
print(vect.get_feature_names_out()[:10])

# least frequent words
print("Top 10 least frequent words in the dataset")
print(vect.get_feature_names_out()[-10:])

Top 10 most frequent words in the dataset
['0' '00' '000' '0000' '000001' '00001' '000013' '0000soo' '0001'
 '000111052']
Top 10 least frequent words in the dataset
['¾' 'â' 'çay' 'çaykur' 'çelem' 'être' 'île' 'ît' 'ø' 'þ']


In [9]:
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

print(X_train[0])

(454763, 111813) (113691, 111813) (454763,) (113691,)
  (0, 88023)	0.2703757583318995
  (0, 99992)	0.35728192737004544
  (0, 57108)	0.6842328554222189
  (0, 29768)	0.37700624873987243
  (0, 24559)	0.19202413658804746
  (0, 74441)	0.28581062226005205
  (0, 65294)	0.2300574892119257
  (0, 62655)	0.13210159908753327


# Recurrent Neural Network
### Model starts here

In [11]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn import metrics
import datetime
import os

2024-05-03 22:08:37.977267: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-03 22:08:38.013212: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-03 22:08:38.463101: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [12]:
def batch_generator(X, y, batch_size=32, num_classes=None, timesteps=1):
    num_samples = X.shape[0]
    features_per_timestep = X.shape[1] // timesteps
    while True:
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            X_batch = X[start:end].toarray()  # Convert only this slice to dense
            X_batch = X_batch.reshape(-1, timesteps, features_per_timestep)  # Reshape for LSTM
            y_batch = to_categorical(y[start:end], num_classes=num_classes)
            yield (X_batch, y_batch)


# Setup the model
model = Sequential([
    LSTM(256, input_shape=(None, X_train.shape[1]), activation='relu', return_sequences=True),
    Dropout(0.2),
    LSTM(128, activation='relu'),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dense(np.max(y_train) + 1, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model using a generator
train_generator = batch_generator(X_train, y_train, batch_size=128, num_classes=np.max(y_train) + 1, timesteps=1)
model.fit(train_generator, epochs=20, steps_per_epoch=int(np.ceil(X_train.shape[0] / 128)))

2024-05-03 22:08:39.189177: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78746 MB memory:  -> device: 0, name: NVIDIA H100 PCIe, pci bus id: 0000:b5:00.0, compute capability: 9.0
/opt/mamba/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20


I0000 00:00:1714774122.328024    6095 service.cc:145] XLA service 0x7f6aec0b8820 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714774122.328337    6095 service.cc:153]   StreamExecutor device (0): NVIDIA H100 PCIe, Compute Capability 9.0
2024-05-03 22:08:42.415908: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-03 22:08:42.833481: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900
I0000 00:00:1714774125.870881    6307 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_59', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1714774127.026194    6387 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_58', 60 bytes spill stores, 60 bytes spill loads

I0000 00:00:1714774127.

   5/3553 ━━━━━━━━━━━━━━━━━━━━ 2:09 36ms/step - accuracy: 0.4352 - loss: 1.7836  

I0000 00:00:1714774131.340207    6095 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3552/3553 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.6997 - loss: 0.8448

I0000 00:00:1714774250.557252    7035 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_59', 4 bytes spill stores, 4 bytes spill loads



3553/3553 ━━━━━━━━━━━━━━━━━━━━ 135s 35ms/step - accuracy: 0.6997 - loss: 0.8448
Epoch 2/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 114s 32ms/step - accuracy: 0.7909 - loss: 0.5777
Epoch 3/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 114s 32ms/step - accuracy: 0.8587 - loss: 0.4018
Epoch 4/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 113s 32ms/step - accuracy: 0.8993 - loss: 0.2869
Epoch 5/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 104s 29ms/step - accuracy: 0.9253 - loss: 0.2114
Epoch 6/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 119s 33ms/step - accuracy: 0.9419 - loss: 0.1649
Epoch 7/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 114s 32ms/step - accuracy: 0.9537 - loss: 0.1313
Epoch 8/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 111s 31ms/step - accuracy: 0.9623 - loss: 0.1068
Epoch 9/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 115s 32ms/step - accuracy: 0.9685 - loss: 0.0897
Epoch 10/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 109s 31ms/step - accuracy: 0.9737 - loss: 0.0769
Epoch 11/20
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 111s 31ms/step - accuracy: 0.9774 - loss: 0.0658
Epoch 12

In [13]:
test_generator = batch_generator(X_test, y_test, batch_size=32, num_classes=np.max(y_train) + 1, timesteps=1)

In [14]:
# Evaluate the model
predictions_prob = model.predict(test_generator, steps=int(np.ceil(X_test.shape[0] / 32)))
predictions = np.argmax(predictions_prob, axis=1)

# Metrics
print(metrics.confusion_matrix(y_test, predictions))
print(metrics.classification_report(y_test, predictions))

3553/3553 ━━━━━━━━━━━━━━━━━━━━ 43s 11ms/step
[[ 7019  1156   495   215  1441]
 [  673  3155   610   346  1071]
 [  363   710  4258  1054  2100]
 [  179   355   701  8682  6206]
 [  378   599   678  2698 68549]]
              precision    recall  f1-score   support

           1       0.82      0.68      0.74     10326
           2       0.53      0.54      0.53      5855
           3       0.63      0.50      0.56      8485
           4       0.67      0.54      0.60     16123
           5       0.86      0.94      0.90     72902

    accuracy                           0.81    113691
   macro avg       0.70      0.64      0.67    113691
weighted avg       0.80      0.81      0.80    113691



In [15]:
# # Save the model with current date and time in model folder

# Create a folder named _models in the current directory
if not os.path.exists('_models'):
    os.makedirs('_models')
    
model.save(f'_models/config1_bis_rnn_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.keras')

In [16]:
# test the model
test_sentences = [
    'This is a good product',
    'This is a bad product',
    'This is a product',
    'This is a very good product',
    'This is a very bad product',
    'That was bad'
]

for sentence in test_sentences:
    test_tokenized = tokenizer([sentence])
    test_vec = vect.transform(test_tokenized)
    test_vec_dense = test_vec.toarray()  # Convert sparse tensor to dense tensor
    test_vec_lstm = test_vec_dense.reshape(-1, 1, X.shape[1])
    result = model.predict(test_vec_lstm)
    predicted_class = np.argmax(result, axis=-1)
    predicted_score = predicted_class
    print(f"Test sentence: {sentence}")
    print(f"Predicted score: {predicted_score}")
    print()

Number of tokens:  2
Number of sentences:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 603ms/step
Test sentence: This is a good product
Predicted score: [5]

Number of tokens:  2
Number of sentences:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Test sentence: This is a bad product
Predicted score: [1]

Number of tokens:  1
Number of sentences:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Test sentence: This is a product
Predicted score: [3]

Number of tokens:  2
Number of sentences:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Test sentence: This is a very good product
Predicted score: [5]

Number of tokens:  2
Number of sentences:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Test sentence: This is a very bad product
Predicted score: [1]

Number of tokens:  1
Number of sentences:  1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Test sentence: That was bad
Predicted score: [5]

